In [1]:
import pandas as pd
import numpy as np
from matplotlib.image import imsave
import os

import random
import matplotlib.pyplot as plt
import matplotlib.animation as animation

%matplotlib inline

# - FUNCTIONS

In [2]:
def remove_negative_coord(array, liste):
    
    array_max = array.shape[0]
    
    liste_without_negative = []
    
    for element in liste:
        if element[0] >= 0 and element[1] >= 0 and element[0] < array_max and element[1] < array_max:
            liste_without_negative.append(element)
            
    return liste_without_negative

In [3]:
def flatten_list_of_list(liste):
    return [item for sublist in liste for item in sublist]

In [4]:
def set_new_patient(array, liste_coord):

    for coord in liste_coord:
        array[coord[0], coord[1]] =1
    
    return array

In [5]:
def neighbors(array, coord, radius=1):
    all_neighbors_coord = []
    
    lst_coord = range(-radius, radius+1)
    
    for x in lst_coord:
        for y in lst_coord:
            if (x != coord[0]) or (y != coord[1]):
                all_neighbors_coord.append([coord[0] + x,coord[1] + y])
                
    all_neighbors_coord = remove_negative_coord(array, all_neighbors_coord)
                
    return all_neighbors_coord

In [6]:
def contamination_qty(liste, percent):

    if percent != 0 :
        n = 100//percent
        return random.sample(liste, len(liste)//n)
    else:
        return liste

In [7]:
def remove_old_patient(array, liste_coord):
    
    liste_coord = flatten_list_of_list(liste_coord)
    
    for coord in liste_coord:
        array[coord[0], coord[1]] = 0
    
    return array

In [8]:
def remove_vacc_patient(array, liste_coord):
    
    for coord in liste_coord:
        array[coord[0], coord[1]] = 0
    
    return array
    

# - SET POPULATION

- All pop

In [44]:
pop_size = 500

pop = np.zeros((pop_size,pop_size), dtype=int)

patient_0_x = np.random.randint(pop_size)
patient_0_y = np.random.randint(pop_size)

patient_0_x = 186
patient_0_y = 336

pop[patient_0_x][patient_0_y] = 1

print('position patient 0 : ({}, {})'.format(patient_0_x, patient_0_y))

position patient 0 : (186, 336)


- Vacc pop

In [45]:
vacc = np.random.choice([0, 1], size=pop_size*pop_size, p=[1, 0]).astype(int)
vacc = np.reshape(vacc, (pop_size, pop_size))
loc_vacc = np.transpose(np.where(vacc==1))


In [46]:
len(loc_vacc)

0

# - ITERATE CONTAMINATION

In [47]:
old_patient = []
all_new_contaminated = []
pop_without_old_patient = pop
qty = 20
folder = 'images/vacc_0_neighbors_divided_by_{}'.format(qty)

os.makedirs(folder, exist_ok=True)

In [48]:
imsave('{}/conta_0.jpg'.format(folder), pop, cmap='binary')

for i in range(99):
    
    all_new_contaminated = []
    location_patient = np.transpose(np.where(pop==1))
    
    for location in location_patient:
        
        local_new_patient = neighbors(pop, location, radius=2)
        local_new_patient = random.sample(local_new_patient, len(local_new_patient)//qty)
        
        all_new_contaminated.append(local_new_patient)
    all_new_contaminated = flatten_list_of_list(all_new_contaminated)
    pop = set_new_patient(pop, all_new_contaminated)
    old_patient.append(location_patient)
    pop = remove_vacc_patient(pop, loc_vacc)
    pop_without_old_patient = remove_old_patient(pop, old_patient)
    
    imsave('{}/conta_{}.jpg'.format(folder, i+1), pop_without_old_patient, cmap='binary')